In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def interpolation(X1,X2,Y1,Y2,XInconnu):
    #prevent division by 0
    if(X2 == X1):
        return Y1
    YInconnu = Y2*((XInconnu-X1)/(X2-X1))+Y1*((X2-XInconnu)/(X2-X1))
    return YInconnu



def interpol_line(line1:pd.core.series.Series, line2:pd.core.series.Series, col2:str, target:int, toInterpol:list[str]=[], toNotInterpol:list[str]=[])->pd.core.series.Series:
    #col2 : colonne de temps 
    #target : temps de la nouvelle ligne
    #les temps doivent être des int ou float, peu importe l'échelle

    x1 = line1[col2]
    x2 = line2[col2]
    res = line1.copy()
    for col in res.index:
        res[col] = np.nan 
    res[col2] = target
    # interpolate all ints in the lines except col2
    for col in line1.index:
        if(col != col2  ):
            #check that col is numeric and not boolean
            if(pd.api.types.is_numeric_dtype(line1[col]) and  not pd.api.types.is_bool_dtype(line1[col])):
                if(col in toInterpol or col not in toNotInterpol):
                    y1 = line1[col]
                    y2 = line2[col]
                    print("interpolating", col, "from", y1, "to", y2, "for", target, "between", x1, "and", x2) 
                    print()
                    res[col]=(interpolation(x1, x2, y1, y2, target))
                    print("result", res[col])
    return res

#(interpol_line(line, line2, 'date_obs', 450))



In [3]:
def normalize(val, min, max):
    return (val-min)/(max-min)
    #FIX : floating point error

def denormalize(val, min, max):
    return val*(max-min)+min

def interpolDf(df, timecol, pas_voulu, offset=0, toInterpol:list[str]=[], toNotInterpol:list[str]=[]):
    #df : dataframe à interpoler
    #timecol : colonne de temps
    #toInterpol : liste des colonnes à interpoler (facultatif)
    #toNotInterpol : liste des colonnes à ne pas interpoler (facultatif)
    #pas_voulu : pas voulu pour l'interpolation
    #offset : décalage à appliquer à la colonne de temps
    #les temps doivent être des int ou float, peu importe l'échelle
  
    #on trie le df par temps
    df = df.sort_values(by=timecol)

    #on récupère les temps min et max
    min = df[timecol].min()
    max = df[timecol].max() 

    #on normalise les temps et on arrondit a 5 chiffres après la virgule
    df[timecol] = df[timecol].apply(lambda x: normalize(x, min, max))
    df[timecol] = df[timecol].apply(lambda x: round(x, 5))

    #on crée une colonne original pour marquer les lignes originales et une colonne filled pour marquer les lignes interpolables
    df['original'] = True
    df['filled'] = True


    #on adapte le pas voulu et l'offset à la normalisation
    pas_voulu = pas_voulu/(max-min)
    offset = offset/(max-min)

    #on crée une liste de temps à interpoler arrondis à 5 chiffres après la virgule
    temps = np.arange(min + offset, 1, pas_voulu)
    temps = [round(t, 5) for t in temps]

    #on ajoute des lignes vides avec les temps à interpoler au df si elles n'existent pas déjà
    for t in temps:  
        if(t not in df[timecol].values): ###NE MARCHE PAS ERREUR PRECISION FLOAT
            newline = pd.Series([np.nan for i in range(len(df.columns))], index=df.columns)
            newline[timecol] = t
            newline['original'] = False 
            newline['filled'] = False
            df = df.append(newline, ignore_index=True)
        
    
    #on trie le df par temps
    df = df.sort_values(by=timecol)
    df = df.reset_index(drop=True)
    print(df)
    #on interpole les lignes vides
    for i in range(len(df)):
        if(df['filled'][i] == False):
            #on récupère les lignes avant et après si elles existent
    
            line1 = None
            line2 = None

            #debug prints
            

            print()
            print('finding line 2')
            for j in range(i+1, 28):
                # print('j',j)
                # print('line j',df.iloc[j])
                if(df['filled'][j] == True):
                    line2 = df.iloc[j]
                    print('line 2 found')
                    break
                # else:
                #     print('line 2 NOT found')
                    
                    
            for k in range(i-1, -1, -1):
                # print('line at k',df.iloc[k])
                if(df['filled'][k] == True):
                    line1 = df.iloc[k]
                    print('line 1 found')
                    break
                # else:
                #     print('line 1 NOT found')
                    
                    

            #on interpole la ligne vide
            # print('LINE1',line1)
            # print('LINE2',line2)
            if(line1 is None or line2 is None):
                print("error : no line before or after line at", i)
                continue
            df.iloc[i] = interpol_line(line1, line2, timecol, df[timecol][i], toInterpol, toNotInterpol)
            df['filled'][i] = True


    #on dénormalise les temps
    df[timecol] = df[timecol].apply(lambda x: denormalize(x, min, max))

    #on supprime les colonne original et filled
    df = df.drop(columns=['original'])
    df = df.drop(columns=['filled'])

    return df

        


In [7]:
#test 
df = pd.read_csv('./weather/testInterpol.csv',sep=';')
df.head()
# df = interpolDf(df, 'date', 1, 2)
print(df)

    date  num1  random  index                   string
0      0     5     0.5      0                Finistère
1      2     7    64.0      1                      Ain
2      4     8     0.5      2                    Aisne
3      6     9     0.5      3                   Allier
4      8    10     0.5      4  Alpes-de-Haute-Provence
5     10    11     0.5      5             Hautes-Alpes
6     12    12     0.5      6          Alpes-Maritimes
7     14    13     0.5      7                  Ardèche
8     16    14     0.5      8                 Ardennes
9     18    15     0.5      9                   Ariège
10    20    16     0.5     10                   Aube  


In [5]:
df.head()

,date,num1,random,index,string
0,0.0,5.0,0.50,0.0,Finistère
1,2.0,7.0,64.00,1.0,Ain
2,3.0,7.5,32.25,1.5,NaN
3,4.0,8.0,0.50,2.0,Aisne
4,5.0,8.5,0.50,2.5,NaN


In [6]:
# approche 1
def intervalle_heure(pas_voulu, dataframe, column, offset=0):
    # pas_base : pas de temps de la base de données
    # pas_voulu : pas de temps voulu
    # offset : décalage de l'heure par rappot a minuit voulu
    # dataframe : dataframe contenant la colonne
    # column : colonne contenant le temps
    # retourne un dataframe avec les lignes correspondant au pas_voulu et les données interpolées
    #les durées sont en minutes

    if not isinstance(dataframe[column][0], pd._libs.tslibs.timestamps.Timestamp):
        raise ValueError("column doit etre de type datetime")
    
    dataframe.sort_values(by=[column], inplace=True)

    #create new binary column "synced" to know if the line is in pas_voulu + offset
    dataframe["synced"] = False

    #loop over days
    for day in dataframe[column].dt.date.unique():
        #loop over lines of the day
        for i in range(offset, 1440-offset, pas_voulu):
            #if there is a line at this time !!! in our reference column ((to implement)))
            if(#line at i 
                 

            )
                dataframe.loc[x, "synced"] = True
            else:
                #get last line before i
                #get first line after i
                #interpolate
                #enter the interpolated data in the dataframe with synced = True
                pass
            #create line with i as time
        
            #get the surrounding lines

            #interpolate to get the data of first line

    #drop all lines that are not synced
    dataframe.drop(dataframe[dataframe["synced"] == 0].index, inplace=True)
    #delete the column synced
    dataframe.drop(columns=["synced"], inplace=True)

    return dataframe




SyntaxError: invalid syntax (988969956.py, line 27)

2 approches : 
1. loop dans 1 jour par le pas voulu
2. loop dans 1 jour par les lignes existantes 

1> permet de creer lignes facilement 

2> permet de recuperer interpolation data facilement


probleme dans le cas ou on a 2 données au meme timestamp

In [ ]:
#2e essai avec 2e ligne de reference pour trier les lignes
# approche 1
def intervalle_heure(pas_voulu, dataframe, timecol,tricol, offset=0):
    # pas_base : pas de temps de la base de données
    # pas_voulu : pas de temps voulu
    # offset : décalage de l'heure par rappot a minuit voulu
    # dataframe : dataframe contenant la colonne
    # timecol : colonne contenant le temps
    # retourne un dataframe avec les lignes correspondant au pas_voulu et les données interpolées
    #les durées sont en minutes

    if not isinstance(dataframe[timecol][0], pd._libs.tslibs.timestamps.Timestamp):
        raise ValueError("timecol doit etre de type datetime")
    
    dataframe.sort_values(by=[tricol,timecol], inplace=True)

    #create new binary column "synced" to know if the line is in pas_voulu + offset
    dataframe["synced"] = False

                                                                                                        #     #store the entire time column in a list
                                                                                                        #     timecol_list = dataframe[timecol].tolist()
                                                                                                        # #### bad idea: time format should be restored after the loop
    # add a minutes column to the dataframe containing minutes since the beginning of the year
    dataframe["minutes"] = dataframe[timecol].dt.hour*60 + dataframe[timecol].dt.minute
     

    #loop over rows with offset+n*pas_voulu while keeping track of the index
    for i in range(offset, 1440-offset, pas_voulu):





       

            #if there is a line at this time !!! in our reference column ((to implement)))
            #line at i 
        
                # dataframe.loc[x, "synced"] = True
            
                #get last line before i
                #get first line after i
                #interpolate
                #enter the interpolated data in the dataframe with synced = True
              
            #create line with i as time
        
            #get the surrounding lines

            #interpolate to get the data of first line

    #drop all lines that are not synced
    dataframe.drop(dataframe[dataframe["synced"] == 0].index, inplace=True)
    #delete the column synced
    dataframe.drop(columns=["synced"], inplace=True)

    return dataframe


In [ ]:
    #create first line with a time of offset
        #get the surrounding lines (get the last line of the day before ?)
        #interpolate to get the data of first line


  if(0): #the line is in pas_voulu + offset
                dataframe["synced"][i] = 1
                continue

In [ ]:
DfOffset = dataframe[dataframe[column].dt.date == day]

In [ ]:
testData = pd.read_csv("testData.csv", sep=";")

In [ ]:
print(interpolation(1,10,15,23,6))